# Crawling [bezdim.org](https://bezdim.org/signali/reports)

### Import Scrapy and install if missing

In [1]:
try:
    import scrapy
except:
    import sys
    !conda install --yes --prefix {sys.prefix} -c conda-forge scrapy
    import scrapy
from scrapy.crawler import CrawlerProcess

#### Other imports

In [2]:
from time import time
from os.path import join
import pandas as pd

timestamp = int(time())

Currently not using the custom pipeline

In [3]:
import json
import codecs


class JsonWriterPipeline:

    def open_spider(self, spider):
        self.file = codecs.open('reportresult.jl', 'w')

    def close_spider(self, spider):
        self.file.close()

    def process_item(self, item, spider):
        line = json.dumps(dict(item)) + "\n"
        self.file.write(line)
        return item

### Helper functions

In [4]:
class ParseUtils:

    @staticmethod
    def parse_description(raw_description):
        striped = [d.strip() for d in raw_description if d.strip()]
        return " ".join(striped)
    
    @staticmethod
    def parse_time_location(when_where_element):
        selectors = when_where_element.xpath('.//span')
        if len(selectors) < 1:
            return ('N/A Data', 'N/A Location')
        
        date = selectors[0].css('span.r_date::text').extract_first(default='N/A Date')
        location = 'N/A Location'
        if len(selectors) > 1:
            location = selectors[1].css('span.r_location::text').extract_first(default='N/A Location')
    
        return date, location
    
    @staticmethod
    def parse_categories(categories_element):
        '''
        Can have multiple categories
        https://bezdim.org/signali/reports/view/10099
        '''
        selector = categories_element.xpath('.//p//a')
        categories = selector.css('a::text').extract()
        
        return [cat.strip() for cat in categories]
    
    @staticmethod
    def parse_file_urls(content_element):
        selector = content_element.xpath('.//ul//li')
        
        files = []
        
        for tag in selector.css('a'):
            url = tag.xpath('@href').extract_first(default='N/A url')
            name = tag.css('::text').extract_first(default='N/A filename')
            files.append({'url':url, 'name':name})
        
        return files

In [5]:
import logging
from scrapy import Spider
from scrapy.http import Request

class BezDimSpider(Spider):
    name = "bezdim"
    start_page = 40
    page_count = 10
    start_urls = [
        'https://bezdim.org/signali/reports'
    ]
    custom_settings = {
        'LOG_LEVEL': logging.WARNING,
#         'ITEM_PIPELINES': {'__main__.JsonWriterPipeline': 1},
        'FEED_FORMAT':'json',
        'FEED_URI': join('reports_data','report-result-{timestamp}.json'.format(timestamp=timestamp))
    }
    
    def start_requests(self):
        for i in range(self.start_page, self.start_page + self.page_count):
            yield Request('{url}/fetch_reports?page={page_id}'.
                          format(url=self.start_urls[0], page_id=i),
                    callback=self.process_page)
    
    def process_page(self, response):
        for link in response.css('a.r_title'):
            url = link.xpath('@href').extract_first()
            yield Request(url, callback=self.parse)
    
    def parse(self, report):
        report_id = int(report.url.split('/')[-1])
        title = report.css('h1.report-title::text').extract_first(default='').strip()
        
        raw_desc = report.css('div.report-description-text::text').extract()
        description = ParseUtils.parse_description(raw_desc)
        
        date, location = ParseUtils.parse_time_location(report.css('p.report-when-where'))
        
        categories = ParseUtils.parse_categories(report.css('div.report-category-list'))
        
        files = ParseUtils.parse_file_urls(report.css('div.content'))
        
        yield {
            'id': report_id,
            'title': title,
            'description': description,
            'date': date,
            'location': location,
            'categories': categories,
            'files': files
        }
        
#             description = report.css('div.r_description::text').extract_first().strip()
#             address = report.css('p.bd_location::text').extract_first().strip()
            
            

In [6]:
process = CrawlerProcess({
    'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'
})

process.crawl(BezDimSpider)
process.start()

2019-01-26 17:11:42 [scrapy.utils.log] INFO: Scrapy 1.5.1 started (bot: scrapybot)
2019-01-26 17:11:42 [scrapy.utils.log] INFO: Versions: lxml 4.2.5.0, libxml2 2.9.8, cssselect 1.0.3, parsel 1.5.1, w3lib 1.20.0, Twisted 18.9.0, Python 3.7.1 (default, Dec 14 2018, 19:28:38) - [GCC 7.3.0], pyOpenSSL 18.0.0 (OpenSSL 1.1.1a  20 Nov 2018), cryptography 2.4.2, Platform Linux-4.19.16-1-MANJARO-x86_64-with-arch-Manjaro-Linux
2019-01-26 17:11:42 [scrapy.crawler] INFO: Overridden settings: {'FEED_FORMAT': 'json', 'FEED_URI': 'reports_data/report-result-1548515502.json', 'LOG_LEVEL': 30, 'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'}


In [7]:
import pandas as pd

df = pd.read_json(
    join('reports_data','report-result-{timestamp}.json'.format(timestamp=timestamp)))

In [8]:
df

,categories,date,description,files,id,location,title
0,[заведение за хранене и развлечение],2016-09-23 09:40:00,Искам да сигнализирам: В ресторант Кошарите се...,[{'url': 'https://bezdim.org/signali/media/upl...,9157,"град София, бул. „Никола Й. Вапцаров“ № 1",Неспазване на забраната за тютюнопушене в рест...
1,[заведение за хранене и развлечение],2016-09-23 20:45:00,име обект: Bricks bar вид обект: Бар град: Вар...,[{'url': 'https://bezdim.org/signali/media/upl...,9152,"град Варна, бул. Сливница № 5","Пушене в Bricks bar, град Варна (мобилен сигнал)"
2,[заведение за хранене и развлечение],2016-09-23 21:32:00,"В заведение ""Хъшове"", което е на ул. Мария Луй...",[{'url': 'https://bezdim.org/signali/media/upl...,9153,"град София, бул. Мария Луйза № 45","Неспазване на забраната в заведение ""Хъшове"", ..."
3,[заведение за хранене и развлечение],2016-09-23 23:45:00,име обект: Bar Tiffany Sofia вид обект: клуб/д...,[{'url': 'https://bezdim.org/signali/media/upl...,9154,"град София, ул. ""Иван Денкоглу"" № 12","Пушене в клуб Тифани, град София (мобилен сиг..."
4,[заведение за хранене и развлечение],2016-09-24 03:38:00,"В клуб ""Джим Бийм"" дават да се пуши в петък и ...",[{'url': 'https://bezdim.org/signali/media/upl...,9158,"град София, ул. Витоша 31-33",Неспазване на забраната за тютюнопушене в клуб...
5,[заведение за хранене и развлечение],2016-09-24 03:56:00,"В новооткрития клуб ПРОВОКАТОР, твърдят че в з...",[{'url': 'https://bezdim.org/signali/media/upl...,9162,"град София, площад Народно събрание",Неспазване на забраната за тютюнопушене в клуб...
6,[заведение за хранене и развлечение],2016-09-07 12:15:00,"Здравейте, днес обядвах в Riverside Restaurant...",[{'url': 'https://bezdim.org/signali/media/upl...,9129,"град Русе, ул. Батак № 3",През лятото пушенето е разрешено в Riverside R...
7,"[работно място, административно учреждение]",2016-09-07 13:26:00,Днес си подавах документи за лична карта в рай...,[{'url': 'https://bezdim.org/signali/media/upl...,9128,"град Варна, ул. Искър № 22","Пушене в сградата на четвърто районно, град Варна"
8,[заведение за хранене и развлечение],2016-06-30 22:18:00,име обект: Mr Pizza вид обект: Ресторант град:...,[{'url': 'https://bezdim.org/signali/media/upl...,9096,"град София, ул. Верила № 3","Пушене в ресторант Mr Pizza, град София (моб..."
9,[заведение за хранене и развлечение],2016-05-21 01:00:00,име обект: Камино вид обект: Пиано бар град: С...,[{'url': 'https://bezdim.org/signali/media/upl...,9058,"град София, ул. Неофит Рилски 70","Пушене в пиано бар Камино, град София (мобилен..."
